In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2013-01-01,2023-12-31&sort=year,asc"

In [ ]:
j = 0
movie_info = []
while True:

    link = requests.get(url)
    soup = BeautifulSoup(link.text, "lxml")

    result = soup.find('div', class_='lister list detail sub-list')
    year = result.find_all('span', class_='lister-item-year text-muted unbold')
    rating = result.find_all('div', class_='inline-block ratings-imdb-rating')
    genre = result.find_all('span', class_='genre')
    title_list = result.find_all('h3')
    director_info = [item.find_all('a', href=True) for item in result.find_all('div', class_='lister-item-content')]

    next_page = soup.find("a", class_="lister-page-next next-page")
    if not next_page:
        break

    np = next_page.get("href")
    cnp = "https://www.imdb.com" + np


    for movie_years, movie_rating, movie_genre, movie_name, movie_directors_name in zip(year, rating, genre, title_list, director_info):
        title = movie_name.find('a')
        directors_list = []
        for dire in movie_directors_name:
            if "/name/" in dire['href']:
                directors_list.append(dire.text)
        director_name = " | ".join(directors_list)
        if " | " in director_name:
            director_name = director_name.split(" | ")[0]
        j = j + 1
        print(f"{j} Title: {title.text.strip()}, Year: {movie_years.text.strip()}, Rating: {movie_rating.text.strip()}, Genre: {movie_genre.text.strip()}, Director: {director_name.strip()}")
        movie_info.append({
            'Title': title,
            'Year': movie_years,
            'Rating': movie_rating,
            'Genre': movie_genre,
            'Director': director_name
        })
    url = cnp



Streaming output truncated to the last 5000 lines.
57967 Title: Age of Stone and Sky: The Sorcerer Beast, Year: (2021), Rating: 6.5, Genre: Fantasy, Director: William L. Cox
57968 Title: Me Vasantrao, Year: (2021), Rating: 9.1, Genre: Biography, Director: Nipun Dharmadhikari
57969 Title: Backwoods Bubba (Full movie), Year: (2021), Rating: 5.6, Genre: Horror, Director: James B. Thomasson
57970 Title: A Spy Movie, Year: (2021), Rating: 8.6, Genre: Comedy, Director: Stephanie Koenig
57971 Title: Barbarian Invasion, Year: (2021), Rating: 6.5, Genre: Action, Drama, Director: Chui Mui Tan
57972 Title: The Christmas Dance, Year: (2021), Rating: 4.6, Genre: Drama, Family, Romance, Director: Michael Arcell
57973 Title: Vellai Yaanai, Year: (2021), Rating: 6.5, Genre: Drama, Director: Subramaniam Shiva
57974 Title: Made in Chinatown, Year: (2021), Rating: 3.9, Genre: Action, Comedy, Director: James Lew
57975 Title: Finlandia, Year: (2021), Rating: 6.5, Genre: Drama, Fantasy, Director: Horacio Al

In [ ]:
import pandas as pd
df = pd.DataFrame(movie_info)
df.to_csv('imdb_movies_info.csv', index=False)

In [ ]:
import pandas as pd

def load_movie_data(csv_file):
    df = pd.read_csv(csv_file)
    return df


def average_imdb_rating(df):
    avg_rating = df['Rating'].mean()
    return avg_rating

def most_common_genre(df):
    genre_counts = df['Genre'].value_counts()
    most_common_genre = genre_counts.idxmax()
    return most_common_genre

def director_with_highest_avg_rating(df):
    director_ratings = df.groupby('Director')['Rating'].mean()
    director_with_highest_rating = director_ratings.idxmax()
    highest_avg_rating = director_ratings.max()
    return director_with_highest_rating, highest_avg_rating

def year_with_highest_number_of_movies(df):
    year_counts = df['Year'].value_counts()
    year_with_highest_count = year_counts.idxmax()
    highest_movie_count = year_counts.max()
    return year_with_highest_count, highest_movie_count

def main():
    csv_file = 'movie_info.csv'
    movie_df = load_movie_data(csv_file)

    avg_rating = average_imdb_rating(movie_df)
    print(f'Average IMDb rating for the top-rated movies: {avg_rating:.2f}')

    common_genre = most_common_genre(movie_df)
    print(f'The most common genre among the top-rated movies: {common_genre}')

    director, rating = director_with_highest_avg_rating(movie_df)
    print(f'Director with the highest average IMDb rating: {director} (Average Rating: {rating:.2f})')

    year, count = year_with_highest_number_of_movies(movie_df)
    print(f'Year with the highest number of top-rated movies: {year} (Number of Movies: {count})')

if __name__ == '__main':
    main()
